In [1]:
import pandas as pd
import numpy as np
from cfbd_api import play_by_play, BIG_TEN_TEAMS

#grabbing all play by play data from the 2022 season for big ten teams
big_ten = play_by_play()


Minnesota
Wisconsin
Ohio State
Penn State
Michigan
Michigan State
Nebraska
Northwestern
Illinois
Indiana
Iowa
Maryland
Purdue
Rutgers


In [2]:
not_big_ten = big_ten[~big_ten.offense.isin(BIG_TEN_TEAMS)]['offense'].unique()
not_big_ten_play_by_play = play_by_play(not_big_ten.tolist())

big_ten = pd.concat([big_ten, not_big_ten_play_by_play])

Eastern Michigan
North Carolina
Louisiana
Buffalo
Washington State
Georgia Southern
Youngstown State
Western Kentucky
Notre Dame
Delaware
East Carolina
UNLV
Bowling Green
Central Michigan
Richmond
Washington
Colorado
Northern Illinois
Louisiana Tech
UTEP
Duke
Toledo
Kansas
Florida Atlantic
Indiana State
Louisville
Akron
Utah State
Iowa State
Western Michigan
Towson
Charlotte
Virginia
Fresno State
Virginia Tech
Syracuse
Temple
Wagner


In [3]:
pd.set_option('mode.chained_assignment', None)

big_ten['yards_gained_percentage'] = big_ten['yards_gained'] / big_ten['distance']

values = [1,1,1,3,4,5,-1,-6,(-((99 - big_ten['yard_line']) * (2.6/99)) - 1.4),6]

conditions = [(big_ten['down'] == 1) & (big_ten['yards_gained_percentage'] >= 0.45) & (big_ten['yards_gained'] < 10) & (~big_ten['play_type'].str.contains('Touchdown')), 
            (big_ten['down'] == 2) & (big_ten['yards_gained_percentage'] >= 0.60) & (big_ten['yards_gained'] < 10) & (~big_ten['play_type'].str.contains('Touchdown')),
            (big_ten['down'] == 3) | (big_ten['down'] == 4) & (big_ten['yards_gained_percentage'] >= 1) & (big_ten['yards_gained'] < 10) & (~big_ten['play_type'].str.contains('Touchdown')),
            (big_ten['yards_gained_percentage'] > 1) & (big_ten['yards_gained'] >= 10) & (big_ten['yards_gained'] < 20) & (~big_ten['play_type'].str.contains('Touchdown')),
            (big_ten['yards_gained_percentage'] > 1) & (big_ten['yards_gained'] >= 20) & (big_ten['yards_gained'] < 40) & (~big_ten['play_type'].str.contains('Touchdown')),
            (big_ten['yards_gained_percentage'] > 1) & (big_ten['yards_gained'] >= 40) & (~big_ten['play_type'].str.contains('Touchdown')),
            (big_ten['yards_gained'] <= -3),
            (big_ten['period'] != 4) & (big_ten['minutes'] > 2) & big_ten['play_type'].str.contains('Interception'),
            (big_ten['play_type'].str.contains('Fumble')),
            (big_ten['play_type'].str.contains('Touchdown'))]

big_ten['success_points'] = np.select(conditions, values, default=0)
big_ten['success_points'].unique()

array([ 0.        ,  1.        , -1.        ,  4.        ,  3.        ,
        5.        ,  6.        , -6.        , -2.16161616, -3.47474747,
       -1.50505051, -2.37171717, -2.79191919, -3.18585859, -2.97575758,
       -2.03030303, -2.76565657, -1.97777778, -2.94949495, -2.55555556,
       -3.55353535, -1.68888889, -2.08282828, -2.24040404, -2.21414141,
       -3.63232323, -1.95151515, -1.76767677, -1.8989899 , -3.78989899,
       -2.05656566, -2.47676768, -1.58383838, -2.45050505, -2.34545455,
       -3.26464646, -2.10909091, -1.92525253, -2.68686869, -1.63636364,
       -2.58181818, -3.71111111, -1.87272727, -2.60808081, -2.13535354,
       -3.08080808, -2.26666667, -3.86868687, -1.79393939, -3.92121212,
       -2.81818182, -3.34343434, -2.92323232, -3.21212121, -3.36969697,
       -3.3959596 , -2.29292929, -2.52929293, -2.18787879, -3.76363636,
       -2.73939394, -2.66060606, -2.5030303 , -1.71515152, -3.84242424])

In [4]:
values = [1, 0]
conditions = [(big_ten['success_points'] <= 0), (big_ten['success_points'] > 0)]

big_ten['defense_stopped'] = np.select(conditions, values)
big_ten['defense_stopped'].head()

0    1
1    0
2    1
3    0
4    1
Name: defense_stopped, dtype: int64

In [5]:
big_ten_defense = big_ten[big_ten['down'] > 0]
big_ten_defense_rush = big_ten_defense[big_ten_defense['play_type'].str.contains('Rush')]
big_ten_defense_pass = big_ten_defense[big_ten_defense['play_type'].str.contains('Pass')]

defense_dvoa_rush = big_ten_defense_rush.groupby(['down', 'yard_line'])['success_points'].agg(['mean', 'count']).reset_index()
defense_dvoa_rush.head()

defense_dvoa_pass = big_ten_defense_pass.groupby(['down', 'yard_line'])['success_points'].agg(['mean', 'count']).reset_index()
defense_dvoa_pass.head()

,down,yard_line,mean,count
0,1,1,2.250000,8
1,1,2,1.714286,7
2,1,3,1.133333,15
3,1,4,2.500000,10
4,1,5,1.000000,14


In [6]:
big_ten_offense = big_ten[big_ten['down'] > 0]
big_ten_offense_rush = big_ten_offense[big_ten_offense['play_type'].str.contains('Rush')]
big_ten_offense_pass = big_ten_offense[big_ten_offense['play_type'].str.contains('Pass')]

offense_dvoa_rush = big_ten_offense_rush.groupby(['down', 'yard_line'])['success_points'].agg(['mean', 'count']).reset_index()
offense_dvoa_rush.head()

offense_dvoa_pass = big_ten_offense_pass.groupby(['down', 'yard_line'])['success_points'].agg(['mean', 'count']).reset_index()
offense_dvoa_pass.head()

,down,yard_line,mean,count
0,1,1,2.250000,8
1,1,2,1.714286,7
2,1,3,1.133333,15
3,1,4,2.500000,10
4,1,5,1.000000,14


In [7]:
from sklearn.preprocessing import StandardScaler
conditions = []
values = []

for row in defense_dvoa_rush.to_dict('records'):
    values.append(row['mean'])
    conditions.append((big_ten_defense_rush["yard_line"] == row['yard_line']) & (big_ten_defense_rush["down"] == row['down']) & (big_ten_defense_rush["play_type"].str.contains('Rush')))

big_ten_defense_rush["success_points_average"] = np.select(conditions, values)
big_ten_defense_rush["voa"] = big_ten_defense_rush["success_points"] / big_ten_defense_rush["success_points_average"]

defense_adjustment = big_ten_defense_rush.groupby(['down', 'distance', 'defense'])['defense_stopped'].agg(['sum','count']).reset_index()
defense_adjustment['percentage'] = defense_adjustment['sum'] / defense_adjustment.groupby(['down', 'distance', 'defense'])["count"].transform('sum')

conditions = []
values = []

for row in defense_adjustment.to_dict('records'):
    values.append(row['percentage'])
    conditions.append((big_ten_defense_rush["distance"] == row['distance']) & (big_ten_defense_rush["down"] == row['down']) & (big_ten_defense_rush["defense"] == row['defense']))

big_ten_defense_rush['defense_stop_percentage'] = np.select(conditions, values)
big_ten_defense_rush['adjusted_voa'] = big_ten_defense_rush['voa'] - big_ten_defense_rush['defense_stop_percentage']
big_ten_defense_rush.replace([np.inf, -np.inf], np.nan, inplace=True)
big_ten_defense_rush.dropna(subset=['adjusted_voa'], how='all', inplace=True)
big_ten_defense_rush = big_ten_defense_rush[big_ten_defense_rush['defense'].isin(BIG_TEN_TEAMS)]
scaler = StandardScaler()
big_ten_defense_rush['dvoa'] = scaler.fit_transform(big_ten_defense_rush[['adjusted_voa']]).flatten()
big_ten_defense_rush[['voa', 'defense_stop_percentage', 'adjusted_voa', 'dvoa']].head()

big_ten_defense_rush_overall_summary = big_ten_defense_rush.groupby(['defense'])['dvoa'].agg(['mean']).reset_index()
big_ten_defense_rush_overall_summary = big_ten_defense_rush_overall_summary.sort_values(by='mean')

big_ten_defense_rush_down_distance_summary = big_ten_defense_rush.groupby(['defense', 'down', 'distance'])['dvoa'].agg(['mean']).reset_index()
big_ten_defense_rush_down_distance_summary = big_ten_defense_rush_down_distance_summary.sort_values(by='mean')
big_ten_defense_rush_overall_summary

,defense,mean
10,Penn State,-0.232968
9,Ohio State,-0.160950
4,Michigan,-0.140684
2,Iowa,-0.140557
7,Nebraska,-0.128054
6,Minnesota,-0.070614
5,Michigan State,-0.014751
12,Rutgers,0.024653
3,Maryland,0.033607
13,Wisconsin,0.058850


In [8]:
big_ten_defense_rush_overall_summary.head()

,defense,mean
10,Penn State,-0.232968
9,Ohio State,-0.160950
4,Michigan,-0.140684
2,Iowa,-0.140557
7,Nebraska,-0.128054


In [9]:
conditions = []
values = []

for row in defense_dvoa_pass.to_dict('records'):
    values.append(row['mean'])
    conditions.append((big_ten_defense_pass["yard_line"] == row['yard_line']) & (big_ten_defense_pass["down"] == row['down']) & (big_ten_defense_pass["play_type"].str.contains('Pass')))

big_ten_defense_pass["success_points_average"] = np.select(conditions, values)
big_ten_defense_pass["voa"] = big_ten_defense_pass["success_points"] / big_ten_defense_pass["success_points_average"]

defense_adjustment = big_ten_defense_pass.groupby(['down', 'distance', 'defense'])['defense_stopped'].agg(['sum','count']).reset_index()
defense_adjustment.loc[:, ('percentage')] = defense_adjustment['sum'] / defense_adjustment.groupby(['down', 'distance', 'defense'])["count"].transform('sum')

conditions = []
values = []

for row in defense_adjustment.to_dict('records'):
    values.append(row['percentage'])
    conditions.append((big_ten_defense_pass["distance"] == row['distance']) & (big_ten_defense_pass["down"] == row['down']) & (big_ten_defense_pass["defense"] == row['defense']))

big_ten_defense_pass['defense_stop_percentage'] = np.select(conditions, values)
big_ten_defense_pass['adjusted_voa'] = big_ten_defense_pass['voa'] - big_ten_defense_pass['defense_stop_percentage']
big_ten_defense_pass.replace([np.inf, -np.inf], np.nan, inplace=True)
big_ten_defense_pass.dropna(subset=['adjusted_voa'], how='all', inplace=True)
big_ten_defense_pass = big_ten_defense_pass[big_ten_defense_pass['defense'].isin(BIG_TEN_TEAMS)]
scaler = StandardScaler()
big_ten_defense_pass['dvoa'] = scaler.fit_transform(big_ten_defense_pass[['adjusted_voa']]).flatten()

big_ten_defense_pass_overall_summary = big_ten_defense_pass.groupby(['defense'])['dvoa'].agg(['mean']).reset_index()
big_ten_defense_pass_overall_summary = big_ten_defense_pass_overall_summary.sort_values(by='mean')


big_ten_defense_pass_down_distance_summary = big_ten_defense_pass.groupby(['defense', 'down', 'distance'])['dvoa'].agg(['mean']).reset_index()
big_ten_defense_pass_down_distance_summary = big_ten_defense_pass_down_distance_summary.sort_values(by='mean')

big_ten_defense_pass_overall_summary

,defense,mean
10,Penn State,-0.185159
2,Iowa,-0.095850
13,Wisconsin,-0.078937
7,Nebraska,-0.072898
9,Ohio State,-0.047253
4,Michigan,-0.043778
12,Rutgers,-0.025708
3,Maryland,-0.014130
1,Indiana,-0.001173
0,Illinois,0.019197


In [10]:
conditions = []
values = []

for row in offense_dvoa_rush.to_dict('records'):
    values.append(row['mean'])
    conditions.append((big_ten_offense_rush["yard_line"] == row['yard_line']) & (big_ten_offense_rush["down"] == row['down']) & (big_ten_offense_rush["play_type"].str.contains('Rush')))

big_ten_offense_rush["success_points_average"] = np.select(conditions, values)
big_ten_offense_rush["voa"] = big_ten_offense_rush["success_points"] / big_ten_offense_rush["success_points_average"]

defense_adjustment = big_ten_offense_rush.groupby(['down', 'distance', 'defense'])['defense_stopped'].agg(['sum','count']).reset_index()
defense_adjustment.loc[:, ('percentage')] = defense_adjustment['sum'] / defense_adjustment.groupby(['down', 'distance', 'defense'])["count"].transform('sum')

conditions = []
values = []

for row in defense_adjustment.to_dict('records'):
    values.append(row['percentage'])
    conditions.append((big_ten_offense_rush["distance"] == row['distance']) & (big_ten_offense_rush["down"] == row['down']) & (big_ten_offense_rush["defense"] == row['defense']))

big_ten_offense_rush['defense_stop_percentage'] = np.select(conditions, values)
big_ten_offense_rush['adjusted_voa'] = big_ten_offense_rush['voa'] - big_ten_offense_rush['defense_stop_percentage']
big_ten_offense_rush.replace([np.inf, -np.inf], np.nan, inplace=True)
big_ten_offense_rush.dropna(subset=['adjusted_voa'], how='all', inplace=True)
big_ten_offense_rush = big_ten_offense_rush[big_ten_offense_rush['offense'].isin(BIG_TEN_TEAMS)]
scaler = StandardScaler()
big_ten_offense_rush['dvoa'] = scaler.fit_transform(big_ten_offense_rush[['adjusted_voa']]).flatten()

big_ten_offense_rush_overall_summary = big_ten_offense_rush.groupby(['offense'])['dvoa'].agg(['mean']).reset_index()
big_ten_offense_rush_overall_summary = big_ten_offense_rush_overall_summary.sort_values(by='mean', ascending=False)

big_ten_offense_rush_down_distance_summary = big_ten_offense_rush.groupby(['offense', 'down', 'distance'])['dvoa'].agg(['mean']).reset_index()
big_ten_offense_rush_down_distance_summary = big_ten_offense_rush_down_distance_summary.sort_values(by='mean', ascending=False)
big_ten_offense_rush_overall_summary

,offense,mean
4,Michigan,0.118670
9,Ohio State,0.112697
7,Nebraska,0.072061
0,Illinois,0.071936
13,Wisconsin,0.067008
3,Maryland,0.027881
6,Minnesota,0.024867
12,Rutgers,0.003259
11,Purdue,-0.000787
10,Penn State,-0.024521


In [11]:
conditions = []
values = []

for row in offense_dvoa_pass.to_dict('records'):
    values.append(row['mean'])
    conditions.append((big_ten_offense_pass["yard_line"] == row['yard_line']) & (big_ten_offense_pass["down"] == row['down']) & (big_ten_offense_pass["play_type"].str.contains('Pass')))

big_ten_offense_pass["success_points_average"] = np.select(conditions, values)
big_ten_offense_pass["voa"] = big_ten_offense_pass["success_points"] / big_ten_offense_pass["success_points_average"]

defense_adjustment = big_ten_offense_pass.groupby(['down', 'distance', 'defense'])['defense_stopped'].agg(['sum','count']).reset_index()
defense_adjustment.loc[:, ('percentage')] = defense_adjustment['sum'] / defense_adjustment.groupby(['down', 'distance', 'defense'])["count"].transform('sum')

conditions = []
values = []

for row in defense_adjustment.to_dict('records'):
    values.append(row['percentage'])
    conditions.append((big_ten_offense_pass["distance"] == row['distance']) & (big_ten_offense_pass["down"] == row['down']) & (big_ten_offense_pass["defense"] == row['defense']))

big_ten_offense_pass['defense_stop_percentage'] = np.select(conditions, values)
big_ten_offense_pass['adjusted_voa'] = big_ten_offense_pass['voa'] - big_ten_offense_pass['defense_stop_percentage']
big_ten_offense_pass.replace([np.inf, -np.inf], np.nan, inplace=True)
big_ten_offense_pass.dropna(subset=['adjusted_voa'], how='all', inplace=True)
big_ten_offense_pass = big_ten_offense_pass[big_ten_offense_pass['offense'].isin(BIG_TEN_TEAMS)]
scaler = StandardScaler()
big_ten_offense_pass['dvoa'] = scaler.fit_transform(big_ten_offense_pass[['adjusted_voa']]).flatten()
big_ten_offense_pass[['voa', 'defense_stop_percentage', 'adjusted_voa', 'dvoa']].head()


big_ten_offense_pass_overall_summary = big_ten_offense_pass.groupby(['offense'])['dvoa'].agg(['mean']).reset_index()
big_ten_offense_pass_overall_summary = big_ten_offense_pass_overall_summary.sort_values(by='mean', ascending=False)

big_ten_offense_pass_down_distance_summary = big_ten_offense_pass.groupby(['offense', 'down', 'distance'])['dvoa'].agg(['mean']).reset_index()
big_ten_offense_pass_down_distance_summary = big_ten_offense_pass_down_distance_summary.sort_values(by='mean', ascending=False)
big_ten_offense_pass_overall_summary


,offense,mean
4,Michigan,0.160522
0,Illinois,0.088964
9,Ohio State,0.058276
7,Nebraska,0.035030
12,Rutgers,0.030188
6,Minnesota,0.024116
3,Maryland,0.021844
5,Michigan State,0.014307
1,Indiana,0.010348
10,Penn State,0.010192


In [12]:

overall_offense = pd.concat([big_ten_offense_pass,big_ten_offense_rush])

overall_offense = overall_offense[overall_offense['offense'].isin(BIG_TEN_TEAMS)]
scaler = StandardScaler()
overall_offense['dvoa'] = scaler.fit_transform(overall_offense[['adjusted_voa']]).flatten()


overall_offense_summary = overall_offense.groupby(['offense'])['dvoa'].agg(['mean']).reset_index()
overall_offense_summary = overall_offense_summary.sort_values(by='mean', ascending=False)
overall_offense_summary

,offense,mean
4,Michigan,0.131202
9,Ohio State,0.087174
0,Illinois,0.081092
7,Nebraska,0.056992
3,Maryland,0.027245
6,Minnesota,0.021751
13,Wisconsin,0.010216
12,Rutgers,0.007004
10,Penn State,-0.012729
5,Michigan State,-0.045360


In [13]:
overall_defense = pd.concat([big_ten_defense_pass,big_ten_defense_rush])

overall_defense = overall_defense[overall_defense['defense'].isin(BIG_TEN_TEAMS)]
scaler = StandardScaler()
overall_defense['dvoa'] = scaler.fit_transform(overall_defense[['adjusted_voa']]).flatten()
overall_defense[['voa', 'defense_stop_percentage', 'adjusted_voa', 'dvoa']].head()


overall_defense_summary = overall_defense.groupby(['defense'])['dvoa'].agg(['mean']).reset_index()
overall_defense_summary = overall_defense_summary.sort_values(by='mean')
overall_defense_summary

,defense,mean
10,Penn State,-0.202217
2,Iowa,-0.118419
9,Ohio State,-0.118122
4,Michigan,-0.098202
7,Nebraska,-0.093361
6,Minnesota,-0.023815
13,Wisconsin,-0.011090
12,Rutgers,0.001183
3,Maryland,0.011607
0,Illinois,0.078561


In [14]:
with pd.ExcelWriter('big_ten_dvoa_week_5.xlsx') as writer:
    overall_offense_summary.to_excel(writer,'overall_offense', index=False)
    big_ten_offense_pass_overall_summary.to_excel(writer,'offense_pass_overall', index=False)
    big_ten_offense_rush_overall_summary.to_excel(writer,'offense_rush_overall', index=False)
    overall_defense_summary.to_excel(writer,'overall_defense', index=False)
    big_ten_defense_pass_overall_summary.to_excel(writer,'defense_pass_overall', index=False)
    big_ten_defense_rush_overall_summary.to_excel(writer,'defense_rush_overall', index=False)
    big_ten_offense_pass_down_distance_summary.to_excel(writer,'offense_pass_down_and_distance', index=False)
    big_ten_offense_rush_down_distance_summary.to_excel(writer,'offense_rush_down_and_distance', index=False)
    big_ten_defense_pass_down_distance_summary.to_excel(writer,'defense_pass_down_and_distance', index=False)
    big_ten_defense_rush_down_distance_summary.to_excel(writer,'defense_rush_down_and_distance', index=False)